In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")




def return_chain(N_chain,value = 1):
    Total = []
    for j in range(N_chain):
        temp=""
        for k in range(N_chain):
            if k==j or k==(j+1)%N_chain:
                temp+="Z"
            else:
                temp+="I"
        Total .append((temp,value))
    return Total


# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

# Antiferro

In [30]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)


def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list(return_chain(4))
    reference_circuit = QuantumCircuit(4)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        4,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(16)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

observable = SparsePauliOp.from_list(return_chain(4))
reference_circuit = QuantumCircuit(4)
reference_circuit.x(0)

variational_form = TwoLocal(
    4,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

[ 0.15577444  1.79779849  0.86238115  0.08334706  2.01902493  1.10946141
  1.96688152  0.73177885  0.32660418 -0.29149005  2.27603349  1.17706414
  1.05671237  1.85061356  2.42042323  0.84496603]
-2.186


# ferro

In [29]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def return_chain(N_chain,value = 1):
    Total = []
    for j in range(N_chain):
        temp=""
        for k in range(N_chain):
            if k==j or k==(j+1)%N_chain:
                temp+="Z"
            else:
                temp+="I"
        Total .append((temp,value))
    return Total

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list(return_chain(4,-1))
    reference_circuit = QuantumCircuit(4)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        4,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(16)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

observable = SparsePauliOp.from_list(return_chain(4,-1))
reference_circuit = QuantumCircuit(4)
reference_circuit.x(0)

variational_form = TwoLocal(
    4,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

[ 2.37801454  0.4059556   0.84727162 -0.30737539  0.91999014  0.0067423
 -0.01119086 -0.06354428  2.95444516 -0.28363901  1.26767572  0.66696357
  0.03043486  0.00930985  0.06525747 -0.05448787]
-3.989
